# probability of the real structure,  under different SHAPE arrays

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)
import sys
sys.path.insert(0,'..')
from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

# Load data

In [32]:
import rna_tools.rna_io as rio

def getdata():
    return rio.get_all_data("../data/RNA16.react","../data/RNA16.dbn")  # {key: rea, seq, stru}

data = getdata()

for k,v in data.items():print k,"\t",  len(v[1])

data for 'tRNA-asp' is corrupted, ignoring...
ADDRSW 	121
GLYCFN 	205
tRNAphe 	76
RNAsep 	154
ZHCV 	95
Z-CIDGMP-1 	135
adenin 	71
23sRNA 	2904
p564 	155
srRNA 	120
ZGLY 	198
MDLOOP 	80
R009 	1542
5srRNA 	170
TRP5 	202


# train predict

In [56]:
import simushape as sim
from rna_tools.rnaplfold import rnaplfold

data = getdata()
model  = sim.make_model(data,['23sRNA','R009'])
data.pop("23sRNA")
data.pop("R009")

predictions = { k: (v[1],sim.predict(model,v[1])) for k,v in data.items()[:3]  }

keys = predictions.keys()[:3]


acc_shaker = {k: rnaplfold(*predictions[k]) for  k in keys} # a predcition is seq,rea



data for 'tRNA-asp' is corrupted, ignoring...


# real and sukosd based accessibility

In [60]:
from rna_tools.sukosd import sukosd
acc_suko = {k: rnaplfold(data[k][1], sukosd(data[k][2]) ) for  k in keys}
acc_shape =  {k: rnaplfold(data[k][1],data[k][0]) for  k in keys}  

In [77]:
from scipy.stats import spearmanr as spear
corr = lambda ac1, ac2:[spear(ac1[k], ac2[k])[0] for k in keys]
corr_suko =  corr(acc_suko, acc_shape)
corr_shaker =  corr(acc_shaker, acc_shape)

# Visualize 

In [78]:
import pandas
import numpy as np
index= ['suko',"shaker"]
data = [corr_suko,corr_shaker]
map(lambda x:x.append(np.mean(x)),data)
pandas.DataFrame(data, columns=keys+['mean'], index=index).T

,suko,shaker
ADDRSW,0.976993,0.983270
GLYCFN,0.294808,0.560019
tRNAphe,0.915173,0.854448
mean,0.728992,0.799245
